In [1]:
# 导入依赖项
import numpy as np
import pandas as pd

In [4]:
# 创建一个ndarray数组
arr = np.array([[1, 2], [3, 4]])
arr

array([[1, 2],
       [3, 4]])

In [5]:
# 创建一个矩阵 - 在numpy中创建矩阵需要使用mat函数，该函数需要输入一个二维的ndarray数组
m = np.mat(arr)
m

matrix([[1, 2],
        [3, 4]])

In [6]:
# 矩阵的转置
m.T

matrix([[1, 3],
        [2, 4]])

In [7]:
# 矩阵的乘法
m * m

matrix([[ 7, 10],
        [15, 22]])

In [8]:
# 获取当前方阵的行列式
np.linalg.det(m)

-2.0000000000000004

In [9]:
# 求解方阵的逆矩阵
m.I

matrix([[-2. ,  1. ],
        [ 1.5, -0.5]])

In [11]:
# 手撕线性回归算法第一步：实现线性回归模型
def linearRegressorModels(dataset):
  
  '''
    构建线性回归模型
  '''
  
  # 1.从数据集中提取特征，构建为一个二维的矩阵
  xMat = np.mat(dataset.iloc[:, : -1].values)